In [ ]:
from typing import Any, Tuple
import yaml
import numpy as np
import itertools

In [ ]:
def sweep_constructor(
    loader: yaml.SafeLoader, node: yaml.nodes.MappingNode
) -> dict[str, Any]:
    values = loader.construct_sequence(node.value[0][1])
    return {"type": "sweep", "values": values}


def coupled_sweep_constructor(
    loader: yaml.SafeLoader, node: yaml.nodes.MappingNode
) -> dict[str, Any]:
    target = node.value[0][1].value
    values = loader.construct_sequence(node.value[1][1])
    return {"target": target, "values": values, "type": "coupled-sweep"}


def get_loader():
    loader = yaml.SafeLoader
    loader.add_constructor("!sweep", sweep_constructor)
    loader.add_constructor("!coupled-sweep", coupled_sweep_constructor)
    return loader

In [ ]:
cfg = yaml.load(open("./config.yaml"), Loader=get_loader())

In [ ]:
cfg

In [ ]:
class Run:
    def __init__(self, config: dict):
        self.config = config
        self.run_configs = []
        sweep_targets = {}
        coupled_targets = {}
        self._cfg_helper(self.config, sweep_targets, coupled_targets)

        print(sweep_targets)
        print(coupled_targets)

        self.run_configs = self._construct_run_configs(sweep_targets, coupled_targets)
        print(self.run_configs)

    def _cfg_helper(
        self, cfg_node: dict, sweep_targets, coupled_targets
    ) -> Tuple[list[Any], list[Any]]:
        for key, node in cfg_node.items():
            if isinstance(node, dict) and "type" in node:
                if node["type"] == "sweep":
                    sweep_targets[key] = {"key": key, "values": node["values"]}
                elif node["type"] == "coupled-sweep":
                    coupled_targets[key] = {
                        "target": node["target"],
                        "values": node["values"],
                    }
            elif isinstance(node, dict):
                self._cfg_helper(node, sweep_targets, coupled_targets)
            else:
                continue  # we ignore leafs

        return sweep_targets, coupled_targets

    def _construct_run_configs(self, sweep_targets, coupled_targets):
        # make list of tuple values
        num_elements = 1
        shape = {}

        return num_elements, shape

In [ ]:
runner = Run(cfg)